# 导入文件

In [5]:
import pandas as pd
import pathlib
base_path = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
excel_file = pd.ExcelFile(base_path)
excel_file.sheet_names

['Sheet1', 'Sheet2', 'Sheet3']

# 自定义函数

In [3]:
%run corelibs.py

# 执行

In [6]:
parse_trans_general(excel_file,'我行账号（卡号）')

[              我行账号（卡号）         我方银行名称 我方中文名称                身份证号  \
 74  035001010101155603  天津银行第二中心支行营业部    赵艳丽  140423198009270829   
 75  035001010101155603  天津银行第二中心支行营业部    赵艳丽  140423198009270829   
 76  035001010101155603  天津银行第二中心支行营业部    赵艳丽  140423198009270829   
 
                    交易日期 对方账号（卡号） 对方中文名称 对方银行名称 对方银行机构代码     流水号  ...  \
 74  2013-12-17 00:00:00      NaN    NaN    NaN      NaN     222  ...   
 75  2014-03-17 00:00:00      NaN    NaN    NaN      NaN  311267  ...   
 76  2014-04-02 00:00:00      NaN    NaN    NaN      NaN  150184  ...   
 
         交易金额   币种        余额 网银IP地址 交易代码（交易渠道） 代码描述 用途（摘要） 受理机构代码 受理机构名称 交易是否成功  
 74  50000.00  CNY  50000.00    NaN        NaN  NaN     批开    NaN    NaN      Y  
 75    357.50  CNY  50357.50    NaN        NaN  NaN     结息    NaN    NaN      Y  
 76  50357.50  CNY      0.00    NaN        NaN  NaN     现销    NaN    NaN      Y  
 
 [3 rows x 21 columns],
               我行账号（卡号）         我方银行名称 我方中文名称                身份证号  \
 72  